In [1]:
import os
import pandas as pd
from uuid import uuid4

In [2]:
# Definição das camadas
dir_silver = "silver"
dir_gold = "gold"
os.makedirs(dir_gold, exist_ok=True)

In [3]:
# Criar diretórios para cada dimensão e fato
os.makedirs(f"{dir_gold}/dim_cliente", exist_ok=True)
os.makedirs(f"{dir_gold}/dim_vendedor", exist_ok=True)
os.makedirs(f"{dir_gold}/dim_produto", exist_ok=True)
os.makedirs(f"{dir_gold}/dim_pagamento", exist_ok=True)
os.makedirs(f"{dir_gold}/dim_avaliacao", exist_ok=True)
os.makedirs(f"{dir_gold}/dim_geolocalizacao", exist_ok=True)
os.makedirs(f"{dir_gold}/fato_pedidos", exist_ok=True)

In [4]:
# Carregar tabelas da camada Silver
customers = pd.read_parquet(f"{dir_silver}/customers")
sellers = pd.read_parquet(f"{dir_silver}/sellers")
products = pd.read_parquet(f"{dir_silver}/products")
product_category_translation = pd.read_parquet(f"{dir_silver}/product_category_name_translation")
order_items = pd.read_parquet(f"{dir_silver}/order_items")
order_payments = pd.read_parquet(f"{dir_silver}/order_payments")
order_reviews = pd.read_parquet(f"{dir_silver}/order_reviews")
orders = pd.read_parquet(f"{dir_silver}/orders")
geolocation = pd.read_parquet(f"{dir_silver}/geolocation")

In [5]:
# Função para gerar UUID
def generate_uuid(df, column_name):
    df[column_name] = [str(uuid4()) for _ in range(len(df))]
    return df

In [6]:
# Criar Dimensão Cliente
dim_cliente = customers.copy()
dim_cliente = generate_uuid(dim_cliente, "customer_sk")
dim_cliente.to_parquet(f"{dir_gold}/dim_cliente/dim_cliente.parquet", index=False)

In [7]:
# Criar Dimensão Vendedor
dim_vendedor = sellers.copy()
dim_vendedor = generate_uuid(dim_vendedor, "seller_sk")
dim_vendedor.to_parquet(f"{dir_gold}/dim_vendedor/dim_vendedor.parquet", index=False)

In [8]:
# Criar Dimensão Produto
dim_produto = products.merge(product_category_translation, on="product_category_name", how="left")
dim_produto = generate_uuid(dim_produto, "product_sk")
dim_produto.to_parquet(f"{dir_gold}/dim_produto/dim_produto.parquet", index=False)

In [9]:
# Criar Dimensão Pagamento
dim_pagamento = order_payments.copy()
dim_pagamento = generate_uuid(dim_pagamento, "payment_sk")
dim_pagamento.to_parquet(f"{dir_gold}/dim_pagamento/dim_pagamento.parquet", index=False)

In [10]:
# Criar Dimensão Avaliação
dim_avaliacao = order_reviews.copy()
dim_avaliacao = generate_uuid(dim_avaliacao, "review_sk")
dim_avaliacao.to_parquet(f"{dir_gold}/dim_avaliacao/dim_avaliacao.parquet", index=False)

In [11]:
# Criar Dimensão Geolocalização
dim_geolocalizacao = geolocation.copy()
dim_geolocalizacao = generate_uuid(dim_geolocalizacao, "geolocation_sk")
dim_geolocalizacao.to_parquet(f"{dir_gold}/dim_geolocalizacao/dim_geolocalizacao.parquet", index=False)

In [12]:
# Criar Fato Pedido
fato_pedidos = orders.merge(order_items, on="order_id", how="left")
fato_pedidos = fato_pedidos.merge(order_payments, on="order_id", how="left")
fato_pedidos = fato_pedidos.merge(order_reviews, on="order_id", how="left")

In [13]:
# Agregações necessárias
fato_pedidos = fato_pedidos.groupby([
    "order_id", "customer_id", "seller_id", "product_id", "order_status",
    "order_purchase_timestamp", "order_approved_at", "order_delivered_carrier_date",
    "order_delivered_customer_date", "order_estimated_delivery_date"
], as_index=False).agg({
    "price": "sum",
    "freight_value": "sum",
    "order_item_id": "count",
    "review_score": "mean",
    "payment_value": "sum",
    "payment_installments": "max"
})

In [14]:
# Renomear colunas agregadas
fato_pedidos.rename(columns={
    "price": "total_price",
    "freight_value": "total_freight_value",
    "order_item_id": "product_count",
    "review_score": "avg_review_score",
    "payment_value": "total_payment_value",
    "payment_installments": "max_payment_installments"
}, inplace=True)

In [15]:
# Merge com dimensões
fato_pedidos = fato_pedidos.merge(dim_cliente[["customer_id", "customer_sk"]], on="customer_id", how="left")
fato_pedidos = fato_pedidos.merge(dim_vendedor[["seller_id", "seller_sk"]], on="seller_id", how="left")
fato_pedidos = fato_pedidos.merge(dim_produto[["product_id", "product_sk"]], on="product_id", how="left")
fato_pedidos = fato_pedidos.merge(dim_pagamento[["order_id", "payment_sk"]], on="order_id", how="left")
fato_pedidos = fato_pedidos.merge(dim_avaliacao[["order_id", "review_sk"]], on="order_id", how="left")

In [16]:
# Salvar tabela fato
fato_pedidos.to_parquet(f"{dir_gold}/fato_pedidos/fato_pedidos.parquet", index=False)